In [14]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

TypeError: tensordot() missing 1 required positional argument: 'axes'

In [2]:
# Build a dataflow graph.
c = tf.constant([[1.0, 2.0], 
                 [3.0, 4.0]])
d = tf.constant([[1.0, 1.0],
                 [0.0, 1.0]])
e = tf.matmul(c, d)

# Construct a `Session` to execute the graph.
sess = tf.Session()

In [3]:
# Execute the graph and store the value that `e` represents in `result`.
result = sess.run(e)

In [4]:
print(result)

[[1. 3.]
 [3. 7.]]


In [5]:
c = tf.constant([0x0000, 0x1212, 0x1012, 0x090a, 0x1212])
# hist = sess.run(tf.bincount(c))
hist = tf.bincount(c)
nonzero_indices = tf.where(tf.not_equal(hist, 0))
print(sess.run((hist, nonzero_indices)))

(array([1, 0, 0, ..., 0, 0, 2], dtype=int32), array([[   0],
       [2314],
       [4114],
       [4626]]))


In [6]:
# 2 dim
c = tf.constant([[0x0000, 0x0000], [0x0002, 0x0002]])
# 1 dim
hist = tf.bincount(c)
# 2 dim (tf.where() returns a Tensor of shape (num_true, rank(condition)), so (2, 1) where '1' holds the index)
nonzero_indices = tf.where(tf.not_equal(hist, 0))
# 2 dim 
nonzero_values = tf.gather(hist, nonzero_indices)
# 1 dim
nonzero_values_squeezed = tf.squeeze(nonzero_values)

h, indices, values, values2 = sess.run((hist, nonzero_indices, nonzero_values, nonzero_values_squeezed))
print(f'hist: {h}, indices: {indices}, values: {values}, values2: {values2}')

hist: [2 0 2], indices: [[0]
 [2]], values: [[2]
 [2]], values2: [2 2]


In [7]:
# 2 dim
c = tf.constant([[0x0000, 0x0000], [0x0002, 0x0002]])
# 1 dim
hist = tf.bincount(c)
# 2 dim (tf.where() returns a Tensor of shape (num_true, rank(condition)), so (2, 1) where '1' holds the index)
nonzero_indices = tf.where(tf.not_equal(hist, 0))
# 2 dim 
nonzero_values_2d = tf.gather(hist, nonzero_indices)
# 1 dim
nonzero_values_1d = tf.squeeze(nonzero_values)
# sparse_indices: 0-D, 1-D, or 2-D Tensor of type int32 or int64.
#      sparse_indices[i] contains the complete index where sparse_values[i] will be placed.
# output_shape: A 1-D Tensor of the same type as sparse_indices.
# sparse_values: A 0-D or 1-D Tensor.
#      Values corresponding to each row of sparse_indices, or a scalar value to be used for all sparse indices.
conf_matrix = tf.sparse_to_dense(nonzero_indices,
                                 (4,),
                                 nonzero_values_1d,
                                 0)
h, indices, values, values2, conf = sess.run(
    (hist, nonzero_indices, nonzero_values_1d, nonzero_values_2d, conf_matrix))
print(f'hist: {h}, indices: {indices}, values: {values}, values2: {values2}, conf_matrix: {conf}')

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
hist: [2 0 2], indices: [[0]
 [2]], values: [2 2], values2: [[2]
 [2]], conf_matrix: [2 0 2 0]


In [8]:
def my_sparse_to_dense(arr, output_shape):
    # 1 dim
    hist = tf.bincount(arr)
    # 2 dim (tf.where() returns a Tensor of shape (num_true, rank(condition)), so (2, 1) where '1' holds the index)
    nonzero_indices = tf.where(tf.not_equal(hist, 0))
    # 2 dim 
    nonzero_values = tf.gather(hist, nonzero_indices)
    # 1 dim
    nonzero_values_1d = tf.squeeze(nonzero_values)
    # 1 dim
    conf_matrix = tf.sparse_to_dense(nonzero_indices,
                                     output_shape,
                                     nonzero_values_1d,
                                     0)

    sess = tf.Session()
    result = sess.run(conf_matrix)
    return result

In [9]:
# output shape must have a dimension of 1 x max
one_d_arr = tf.constant([0x0000, 0x0000, 0x0002, 0x0002, 0x1212])
print(my_sparse_to_dense(one_d_arr, (1*16*16*16 + 2*16*16 + 1*16 + 2 + 1,)))

[2 0 2 ... 0 0 1]


In [10]:
two_d_arr = tf.constant([[0x0000, 0x0003], [0x0002, 0x0002]])
print(my_sparse_to_dense(two_d_arr, (4,)))

[1 0 2 1]


In [11]:
# if one dimension array of the max value Ox1212 (19 values cross ref), 
# the output shape of the dense array needs to be 1d array of len 65536 (=256x256)
two_d_arr = tf.constant([[0x0102, 0x0201], [0x0002, 0x1212]])
one_d_dense = my_sparse_to_dense(two_d_arr, (0xFFFF + 1,))
one_d_dense

array([0, 0, 1, ..., 0, 0, 0], dtype=int32)

In [12]:
# then, one_d_dense array can be reshaped to 2d_dense array of (256, 256)
two_d_dense = one_d_dense.reshape((0xFF + 1, 0xFF + 1))
two_d_dense

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)